In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -U transformers datasets peft accelerate bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch

model_path = "google/gemma-3-1b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="eager",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,522,880 || all params: 1,006,408,832 || trainable%: 0.6481


In [ ]:
from datasets import Dataset
import json

with open("/content/train.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

dataset = Dataset.from_list(data)

for i in range(min(3, len(dataset))):
    print(json.dumps(dataset[i], indent=2, ensure_ascii=False))


{
  "messages": [
    {
      "content": "Saya siap kapanpun anda siap!",
      "role": "user"
    },
    {
      "content": "Hehe, semangatnya luar biasa! 😄 Saya siap 24 jam, jadi kapan pun kamu butuh bantuan, tinggal bilang saja. Mau mulai sekarang? Ada topik tertentu yang ingin kamu bahas atau butuh bantuan? 🚀",
      "role": "assistant"
    },
    {
      "content": "Samudera ama laut bedanya apa?",
      "role": "user"
    },
    {
      "content": "Pertanyaan yang bagus! Banyak orang sering menganggap *samudera* dan *laut* itu sama, padahal ada perbedaan penting antara keduanya. Yuk, kita bahas:\n\n\n### 🔹 Laut (Sea)\n- **Ukuran lebih kecil** dibanding samudera.\n- **Berada di tepi benua**, biasanya dikelilingi oleh daratan sebagian (tidak sepenuhnya terbuka).\n- **Lebih dangkal** dibanding samudera.\n- Sering **terhubung dengan samudera**, tapi dipisahkan oleh daratan atau gundukan dasar laut.\n- **Contoh**: Laut Jawa, Laut Cina Selatan, Laut Mediterania, Laut Hitam.\n\n\n### 🔹 

In [ ]:
from transformers import DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(
        tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        ),
        truncation=True,
        max_length=4096,
    )

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

lengths = [
    len(tokenizer.apply_chat_template(x["messages"], tokenize=True))
    for x in dataset
]
print("Max length:", max(lengths))
print("Avg length:", sum(lengths) // len(lengths))
print(tokenizer.apply_chat_template(dataset[0]["messages"], tokenize=False))


Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

Max length: 2579
Avg length: 395
<bos><start_of_turn>user
Saya siap kapanpun anda siap!<end_of_turn>
<start_of_turn>model
Hehe, semangatnya luar biasa! 😄 Saya siap 24 jam, jadi kapan pun kamu butuh bantuan, tinggal bilang saja. Mau mulai sekarang? Ada topik tertentu yang ingin kamu bahas atau butuh bantuan? 🚀<end_of_turn>
<start_of_turn>user
Samudera ama laut bedanya apa?<end_of_turn>
<start_of_turn>model
Pertanyaan yang bagus! Banyak orang sering menganggap *samudera* dan *laut* itu sama, padahal ada perbedaan penting antara keduanya. Yuk, kita bahas:


### 🔹 Laut (Sea)
- **Ukuran lebih kecil** dibanding samudera.
- **Berada di tepi benua**, biasanya dikelilingi oleh daratan sebagian (tidak sepenuhnya terbuka).
- **Lebih dangkal** dibanding samudera.
- Sering **terhubung dengan samudera**, tapi dipisahkan oleh daratan atau gundukan dasar laut.
- **Contoh**: Laut Jawa, Laut Cina Selatan, Laut Mediterania, Laut Hitam.


### 🔹 Samudera (Ocean)
- **Jauh lebih luas dan dalam**.
- **Tidak d

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import TrainingArguments, Trainer

model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    warmup_ratio=0.1,
    learning_rate=2e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_dir="logs",
    logging_steps=40,
    save_steps=200,
    save_total_limit=1,
    report_to="none",
    fp16=True,
    group_by_length=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-2088911655.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
40,3.720600
80,2.494200
120,2.263800
160,2.232000
200,2.073300
240,2.063600
280,2.083200
320,2.051400
360,1.885000
400,1.897200


TrainOutput(global_step=480, training_loss=2.213650925954183, metrics={'train_runtime': 2558.9129, 'train_samples_per_second': 1.501, 'train_steps_per_second': 0.188, 'total_flos': 6433611778881792.0, 'train_loss': 2.213650925954183, 'epoch': 3.0})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="eager",
    trust_remote_code=True
)

lora_model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/output/checkpoint-480",
    is_trainable=False
)

merged_model = lora_model.merge_and_unload()

save_path = "/content/drive/MyDrive/Sorachio-1B-4096-2e-4"
merged_model.save_pretrained(save_path, safe_serialization=True)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/Sorachio-1B-4096-2e-4/tokenizer_config.json',
 '/content/drive/MyDrive/Sorachio-1B-4096-2e-4/special_tokens_map.json',
 '/content/drive/MyDrive/Sorachio-1B-4096-2e-4/chat_template.jinja',
 '/content/drive/MyDrive/Sorachio-1B-4096-2e-4/tokenizer.model',
 '/content/drive/MyDrive/Sorachio-1B-4096-2e-4/added_tokens.json',
 '/content/drive/MyDrive/Sorachio-1B-4096-2e-4/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/Sorachio-1B-4096-2e-4"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="eager"
).eval()

messages = [
    {"role": "user", "content": "Perkenalkan dirimu"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=(input_ids != tokenizer.pad_token_id).long(),
        max_new_tokens=256,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

output_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(output_text)


Halo! Aku Sorachio, asisten AI yang dikembangkan oleh Idle Labs.
Tujuan utama diriku adalah membantu kamu dengan cara yang ramah, informatif, dan menyenangkan.
Aku bisa jawab pertanyaan, bantu brainstorming, atau sekadar ngobrol santai.
Jadi, jangan ragu untuk ajak aku ngobrol — aku seneng nemuin teman ngobrol baru 😄
